# Topic Modeling

## Introduction

Another popular text analysis technique is called topic modeling. The ultimate goal of topic modeling is to find various topics that are present in your corpus. Each document in the corpus will be made up of at least one topic, if not multiple topics.

In this notebook, we will be covering the steps on how to do **Latent Dirichlet Allocation (LDA)**, which is one of many topic modeling techniques. It was specifically designed for text data.

To use a topic modeling technique, you need to provide (1) a document-term matrix and (2) the number of topics you would like the algorithm to pick up.

Once the topic modeling technique is applied, your job as a human is to interpret the results and see if the mix of words in each topic make sense. If they don't make sense, you can try changing up the number of topics, the terms in the document-term matrix, model parameters, or even try a different model.

## Topic Modeling - Attempt #1 (All Text)

In [37]:
# Let's read in our document-term matrix
import pandas as pd
import pickle

data = pd.read_pickle('dtm_stop.pkl')
data

,1like,1minutes,1seconds,57,6hbo,72,7jenny,aaaaah,aaah,aaahassold,...,zombie,zombies,zoning,zoo,zoom,zuck,zuckerbergs,zuzus,éclair,éponine
ahir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alex,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ali,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,0,0,0,0,0,1,0,0,...,1,1,1,0,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
hasan,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
jacqueline,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1


In [4]:
# Import the necessary modules for LDA with gensim
# Terminal / Anaconda Navigator: conda install -c conda-forge gensim
from gensim import matutils, models
import scipy.sparse

# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [5]:
# One of the required inputs is a term-document matrix
tdm = data.transpose()
tdm.head(20)

,ahir,alex,ali,anthony,bill,bo,dave,george,hasan,jacqueline,...,jim,joe,john,keith,louis,mark,michelle,mike,patton,ricky
1like,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1minutes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1seconds,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
57,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6hbo,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7jenny,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
aaaaah,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaah,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aaahassold,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [6]:
# We're going to put the term-document matrix into a new gensim format, from df --> sparse matrix --> gensim corpus
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
corpus

In [7]:
# Gensim also requires dictionary of the all terms and their respective location in the term-document matrix
cv = pickle.load(open("cv_stop.pkl", "rb"))
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term), we need to specify two other parameters - the number of topics and the number of passes. Let's start the number of topics at 2, see if the results make sense, and increase the number from there.

In [8]:
# Now that we have the corpus (term-document matrix) and id2word (dictionary of location: term),
# we need to specify two other parameters as well - the number of topics and the number of passes
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=2, passes=10)
lda.print_topics()

[(0,
  '0.026*"like" + 0.016*"know" + 0.015*"im" + 0.014*"just" + 0.012*"dont" + 0.008*"thats" + 0.007*"right" + 0.007*"youre" + 0.007*"said" + 0.006*"people"'),
 (1,
  '0.028*"like" + 0.012*"know" + 0.011*"right" + 0.011*"im" + 0.011*"just" + 0.009*"dont" + 0.007*"thats" + 0.006*"said" + 0.006*"youre" + 0.006*"people"')]

In [9]:
# LDA for num_topics = 3
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=3, passes=10)
lda.print_topics()

[(0,
  '0.011*"right" + 0.011*"like" + 0.006*"just" + 0.004*"think" + 0.004*"thats" + 0.004*"know" + 0.003*"time" + 0.003*"man" + 0.003*"way" + 0.003*"country"'),
 (1,
  '0.030*"like" + 0.017*"know" + 0.016*"im" + 0.016*"just" + 0.013*"dont" + 0.009*"right" + 0.009*"thats" + 0.008*"youre" + 0.007*"people" + 0.006*"got"'),
 (2,
  '0.026*"like" + 0.012*"im" + 0.011*"said" + 0.011*"know" + 0.009*"dont" + 0.008*"just" + 0.006*"right" + 0.006*"thats" + 0.006*"people" + 0.006*"got"')]

In [10]:
# LDA for num_topics = 4
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=4, passes=10)
lda.print_topics()

[(0,
  '0.036*"like" + 0.016*"im" + 0.014*"just" + 0.014*"know" + 0.011*"dont" + 0.010*"said" + 0.009*"right" + 0.007*"thats" + 0.007*"got" + 0.006*"youre"'),
 (1,
  '0.028*"like" + 0.014*"know" + 0.014*"im" + 0.014*"just" + 0.012*"dont" + 0.010*"right" + 0.009*"thats" + 0.007*"youre" + 0.007*"people" + 0.006*"think"'),
 (2,
  '0.018*"know" + 0.015*"like" + 0.013*"im" + 0.011*"just" + 0.010*"dont" + 0.008*"said" + 0.007*"thats" + 0.006*"youre" + 0.005*"right" + 0.005*"want"'),
 (3,
  '0.001*"like" + 0.001*"just" + 0.000*"know" + 0.000*"im" + 0.000*"dont" + 0.000*"right" + 0.000*"thats" + 0.000*"youre" + 0.000*"gonna" + 0.000*"said"')]

Topic 0: Conversational Phrases and Personal Opinions

Topic 1: Everyday Speech and Social Interactions

Topic 2: Expressing Thoughts and Intentions

Topic 3: General Filler Language

These topics aren't looking too great. We've tried modifying our parameters. Let's try modifying our terms list as well.

## Topic Modeling - Attempt #2 (Nouns Only)

One popular trick is to look only at terms that are from one part of speech (only nouns, only adjectives, etc.). Check out the UPenn tag set: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html.

In [11]:
import nltk
nltk.data.path.append('D:/NaturalLanguageProcessing/nltk_data')  # Or another directory you prefer
nltk.download('punkt')  # For word tokenization
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')  # For POS tagging
nltk.download('averaged_perceptron_tagger_eng')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CGDXK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\CGDXK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\CGDXK\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\CGDXK\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [12]:
# Let's create a function to pull out nouns from a string of text
from nltk import word_tokenize, pos_tag

def nouns(text):
    '''Given a string of text, tokenize the text and pull out only the nouns.'''
    is_noun = lambda pos: pos[:2] == 'NN'
    tokenized = word_tokenize(text)
    all_nouns = [word for (word, pos) in pos_tag(tokenized) if is_noun(pos)] 
    return ' '.join(all_nouns)

In [13]:
# Read in the cleaned data, before the CountVectorizer step
data_clean = pd.read_pickle('corpus.pkl')
data_clean

,transcript,full_name
ahir,comedy ahir shah ends transcript septemberfrom...,Ahir Shah
alex,comedy alex edelman just for us transcript apr...,Dave Chappelle
ali,comedy ali wong baby cobra transcript septembe...,Ricky Gervais
anthony,comedy anthony jeselnik thoughts and prayers– ...,Bo Burnham
bill,comedy bill burr im sorry you feel that way– f...,Bill Burr
bo,this page doesnt seem to exist it looks like t...,Jim Gaffigan
dave,comedy dave chappelle the age of spin– transcr...,John Mulaney
george,comedy george carlin im glad im dead transcrip...,Ali Wong
hasan,comedy hasan minhaj off with his head transcri...,Anthony Jeselnik
jacqueline,comedy jacqueline novak get on your knees tran...,Mike Birbiglia


In [14]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns = pd.DataFrame(data_clean.transcript.apply(nouns))
data_nouns

,transcript
ahir,comedy ahir transcript londons court theatre a...
alex,comedy edelman alex show experience meeting na...
ali,comedy ali baby cobra transcript wongs delves ...
anthony,comedy anthony jeselnik thoughts transcript th...
bill,comedy bill sorry transcript mayall thank than...
bo,page doesnt link search
dave,comedy age transcript age spin hollywood chapp...
george,comedy george carlin im im transcript januaryg...
hasan,comedy hasan head transcript minhaj delivers s...
jacqueline,comedy jacqueline knees knees jacqueline stand...


In [15]:
# Create a new document-term matrix using only nouns
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer

# Re-add the additional stop words since we are recreating the document-term matrix
add_stop_words = ['like', 'im', 'know', 'just', 'dont', 'thats', 'right', 'people',
                  'youre', 'got', 'gonna', 'time', 'think', 'yeah', 'said']
stop_words = list(text.ENGLISH_STOP_WORDS.union(add_stop_words))

# Recreate a document-term matrix with only nouns
cvn = CountVectorizer(stop_words=stop_words)
data_cvn = cvn.fit_transform(data_nouns.transcript)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names_out())
data_dtmn.index = data_nouns.index
data_dtmn

,aaahassold,aah,aback,abba,abc,abcs,abdomen,abenefit,abercrombie,abigail,...,zip,zipper,zombie,zombies,zoo,zoom,zuck,zuckerbergs,zuzus,éclair
ahir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alex,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ali,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,0,0,0,0,0,1,0,1,0,0,...,0,0,1,1,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
jacqueline,0,0,0,0,0,0,1,0,2,1,...,0,1,0,0,0,0,0,0,1,0


In [16]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn.transpose()))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())

In [17]:
# Let's start with 2 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=2, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"man" + 0.010*"shit" + 0.007*"guy" + 0.006*"thing" + 0.006*"way" + 0.006*"lot" + 0.006*"life" + 0.006*"fuck" + 0.006*"cause" + 0.006*"day"'),
 (1,
  '0.008*"way" + 0.008*"thing" + 0.006*"day" + 0.006*"cause" + 0.006*"hes" + 0.006*"things" + 0.005*"women" + 0.005*"guy" + 0.005*"life" + 0.005*"man"')]

In [18]:
# Let's try topics = 3
ldan = models.LdaModel(corpus=corpusn, num_topics=3, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.009*"man" + 0.008*"way" + 0.008*"thing" + 0.008*"shit" + 0.007*"guy" + 0.007*"day" + 0.007*"cause" + 0.006*"hes" + 0.006*"gon" + 0.006*"fuck"'),
 (1,
  '0.006*"thing" + 0.005*"day" + 0.005*"things" + 0.005*"way" + 0.005*"id" + 0.004*"years" + 0.004*"point" + 0.004*"kind" + 0.004*"world" + 0.004*"hes"'),
 (2,
  '0.008*"women" + 0.008*"man" + 0.008*"way" + 0.007*"lot" + 0.007*"thing" + 0.007*"cause" + 0.006*"guy" + 0.006*"life" + 0.006*"hes" + 0.005*"day"')]

In [19]:
# Let's try 4 topics
ldan = models.LdaModel(corpus=corpusn, num_topics=4, id2word=id2wordn, passes=10)
ldan.print_topics()

[(0,
  '0.010*"thing" + 0.009*"women" + 0.009*"day" + 0.008*"guy" + 0.008*"cause" + 0.007*"life" + 0.007*"way" + 0.007*"lot" + 0.007*"things" + 0.007*"hes"'),
 (1,
  '0.011*"way" + 0.008*"thing" + 0.007*"hes" + 0.007*"cause" + 0.007*"job" + 0.006*"blow" + 0.006*"life" + 0.006*"man" + 0.006*"guy" + 0.005*"penis"'),
 (2,
  '0.006*"way" + 0.005*"shit" + 0.005*"world" + 0.004*"lot" + 0.004*"dad" + 0.004*"george" + 0.004*"carlin" + 0.004*"theyre" + 0.003*"money" + 0.003*"years"'),
 (3,
  '0.015*"man" + 0.011*"shit" + 0.009*"fuck" + 0.006*"god" + 0.006*"gon" + 0.006*"lot" + 0.005*"cause" + 0.005*"guy" + 0.005*"jamie" + 0.004*"way"')]

Topic 0: Daily Life and Gender Perspectives

Topic 1: Work, Masculinity, and Social Roles

Topic 2: Society, Family, and Commentary

Topic 3: Strong Language and Personal Expression

## Topic Modeling - Attempt #3 (Nouns and Adjectives)

In [20]:
# Let's create a function to pull out nouns from a string of text
def nouns_adj(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and adjectives.'''
    is_noun_adj = lambda pos: pos[:2] == 'NN' or pos[:2] == 'JJ'
    tokenized = word_tokenize(text)
    nouns_adj = [word for (word, pos) in pos_tag(tokenized) if is_noun_adj(pos)] 
    return ' '.join(nouns_adj)

In [21]:
# Apply the nouns function to the transcripts to filter only on nouns
data_nouns_adj = pd.DataFrame(data_clean.transcript.apply(nouns_adj))
data_nouns_adj

,transcript
ahir,comedy ahir transcript septemberfrom londons c...
alex,comedy alex edelman aprilstandup comic alex sh...
ali,comedy ali wong baby cobra transcript septembe...
anthony,comedy anthony jeselnik thoughts full transcri...
bill,comedy bill im sorry full transcript mayall th...
bo,page doesnt link faulty search
dave,comedy age spin– transcript aprilthe age spin ...
george,comedy george carlin im glad im dead transcrip...
hasan,comedy hasan head transcript octoberhasan minh...
jacqueline,comedy jacqueline novak knees knees jacqueline...


In [22]:
# Create a new document-term matrix using only nouns and adjectives, also remove common words with max_df
cvna = CountVectorizer(stop_words=stop_words, max_df=.8)
data_cvna = cvna.fit_transform(data_nouns_adj.transcript)
data_dtmna = pd.DataFrame(data_cvna.toarray(), columns=cvna.get_feature_names_out())
data_dtmna.index = data_nouns_adj.index
data_dtmna

,aaaaah,aaahassold,aah,aback,abba,abc,abcs,abdomen,abenefit,abercrombie,...,zip,zipper,zombie,zombies,zoo,zoom,zuck,zuckerbergs,zuzus,éclair
ahir,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
alex,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
ali,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
anthony,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bill,1,0,0,0,0,0,1,0,1,0,...,0,0,1,1,0,0,0,0,0,0
bo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dave,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
george,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
hasan,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
jacqueline,0,0,0,0,0,0,0,1,0,2,...,0,1,0,0,0,0,0,0,2,0


In [23]:
# Create the gensim corpus
corpusna = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmna.transpose()))

# Create the vocabulary dictionary
id2wordna = dict((v, k) for k, v in cvna.vocabulary_.items())

In [24]:
# Let's start with 2 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=2, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.005*"fuck" + 0.004*"joke" + 0.003*"friend" + 0.003*"mom" + 0.002*"jewish" + 0.002*"course" + 0.002*"okay" + 0.002*"parents" + 0.002*"baby" + 0.002*"fucking"'),
 (1,
  '0.004*"fuck" + 0.004*"job" + 0.003*"blow" + 0.003*"penis" + 0.003*"fucking" + 0.003*"fck" + 0.003*"dude" + 0.003*"uh" + 0.003*"okay" + 0.003*"sort"')]

In [25]:
# Let's try 3 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=3, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"jamie" + 0.004*"foxx" + 0.004*"fuck" + 0.003*"george" + 0.003*"everybody" + 0.003*"carlin" + 0.003*"fck" + 0.003*"dead" + 0.002*"husband" + 0.002*"ngga"'),
 (1,
  '0.005*"job" + 0.005*"fuck" + 0.004*"blow" + 0.004*"penis" + 0.003*"uh" + 0.003*"sort" + 0.003*"fucking" + 0.003*"okay" + 0.003*"fck" + 0.002*"joke"'),
 (2,
  '0.005*"fuck" + 0.003*"fucking" + 0.003*"mom" + 0.003*"dad" + 0.003*"joke" + 0.003*"okay" + 0.003*"jewish" + 0.003*"course" + 0.003*"parents" + 0.003*"dude"')]

In [26]:
# Let's try 4 topics
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=10)
ldana.print_topics()

[(0,
  '0.006*"joke" + 0.003*"dead" + 0.003*"jokes" + 0.003*"news" + 0.003*"job" + 0.002*"jenny" + 0.002*"george" + 0.002*"anthony" + 0.002*"rich" + 0.002*"carlin"'),
 (1,
  '0.006*"job" + 0.006*"blow" + 0.005*"penis" + 0.005*"fuck" + 0.004*"dude" + 0.004*"fck" + 0.003*"everybody" + 0.003*"gay" + 0.003*"fucking" + 0.003*"dick"'),
 (2,
  '0.009*"fuck" + 0.005*"jamie" + 0.004*"fucking" + 0.004*"stroke" + 0.003*"dad" + 0.003*"ass" + 0.003*"foxx" + 0.003*"uh" + 0.003*"country" + 0.003*"son"'),
 (3,
  '0.005*"jewish" + 0.005*"mom" + 0.003*"baby" + 0.003*"okay" + 0.003*"wife" + 0.003*"clinton" + 0.003*"friend" + 0.003*"parents" + 0.003*"dad" + 0.003*"uh"')]

Topic 0: Comedy and Comedians

Topic 1: Explicit Language and Informal Speech

Topic 2: Strong Language and Personal References

Topic 3: Family and Social Relations

## Identify Topics in Each Document

Out of the 9 topic models we looked at, the nouns and adjectives, 4 topic one made the most sense. So let's pull that down here and run it through some more iterations to get more fine-tuned topics.

In [38]:
# Our final LDA model (for now)
ldana = models.LdaModel(corpus=corpusna, num_topics=4, id2word=id2wordna, passes=100)
ldana.print_topics()

[(0,
  '0.004*"fuck" + 0.004*"uh" + 0.003*"fucking" + 0.003*"dude" + 0.003*"everybody" + 0.003*"gay" + 0.003*"hard" + 0.003*"country" + 0.002*"sex" + 0.002*"love"'),
 (1,
  '0.007*"job" + 0.007*"blow" + 0.006*"penis" + 0.004*"fck" + 0.004*"jamie" + 0.004*"fuck" + 0.003*"dick" + 0.003*"sort" + 0.003*"okay" + 0.003*"foxx"'),
 (2,
  '0.008*"fuck" + 0.004*"dad" + 0.004*"stroke" + 0.003*"door" + 0.003*"boundaries" + 0.003*"gay" + 0.002*"mom" + 0.002*"news" + 0.002*"dog" + 0.002*"parents"'),
 (3,
  '0.007*"joke" + 0.004*"baby" + 0.004*"mom" + 0.004*"jewish" + 0.004*"fucking" + 0.004*"okay" + 0.003*"dad" + 0.003*"fuck" + 0.003*"children" + 0.003*"parents"')]

These four topics look pretty decent. Let's settle on these for now.
Topic 0: Explicit Language and Social Commentary

Topic 1: Sexual Humor and Celebrity References

Topic 2: Family and Difficult Situations

Topic 3: Jokes and Family Relationships

In [28]:
corpus_transformed = list(ldana[corpusna])  # Convert generator to list
dominant_topics = [max(doc, key=lambda x: x[1])[0] for doc in corpus_transformed]
result = list(zip(dominant_topics, data_dtmna.index))

## Additional Exercises

1. Try further modifying the parameters of the topic models above and see if you can get better topics.
2. Create a new topic model that includes terms from a different [part of speech](https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html) and see if you can get better topics.

In [29]:
from gensim import models

# Modifying parameters for better topic extraction
lda = models.LdaModel(corpus=corpusna, 
                      id2word=id2word, 
                      num_topics=5,  # Try more topics
                      passes=80,     # Increase passes for better convergence
                      alpha='auto',  # Set alpha to 'auto' to improve model fitting
                      eta=0.01)     # Reduce eta for more specific topic-word associations

lda.print_topics()

[(0,
  '0.011*"bang" + 0.010*"gehrig" + 0.010*"knowjust" + 0.009*"double" + 0.007*"gagi" + 0.005*"oheveryone" + 0.005*"countrybut" + 0.005*"doctors" + 0.005*"dowhat" + 0.004*"percent"'),
 (1,
  '0.009*"double" + 0.007*"desert" + 0.006*"generously" + 0.004*"dowhat" + 0.004*"annoy" + 0.004*"arielat" + 0.004*"overpriced" + 0.004*"alls" + 0.004*"daylight" + 0.004*"enjoying"'),
 (2,
  '0.010*"nationalists" + 0.007*"att" + 0.007*"drown" + 0.007*"events" + 0.007*"jackedup" + 0.007*"protein" + 0.006*"guard" + 0.006*"consumed" + 0.006*"loyalty" + 0.005*"boband"'),
 (3,
  '0.006*"checked" + 0.005*"airconditioned" + 0.004*"dumpif" + 0.004*"double" + 0.004*"body" + 0.004*"desert" + 0.004*"daylight" + 0.004*"mention" + 0.003*"answeri" + 0.003*"briefcase"'),
 (4,
  '0.005*"hump" + 0.004*"kidheavy" + 0.004*"clause" + 0.004*"gaymoremark" + 0.004*"cheese" + 0.003*"generously" + 0.003*"jackedup" + 0.003*"double" + 0.003*"arielat" + 0.003*"briefcase"')]

In [30]:
from nltk import word_tokenize, pos_tag

def nouns_and_verbs(text):
    '''Given a string of text, tokenize the text and pull out only the nouns and verbs.'''
    is_noun_or_verb = lambda pos: pos[:2] in ['NN', 'VB','JJ']
    tokenized = word_tokenize(text)
    all_nouns_verbs = [word for (word, pos) in pos_tag(tokenized) if is_noun_or_verb(pos)]
    return ' '.join(all_nouns_verbs)

In [31]:
# Apply the function to extract nouns and verbs
data_nouns_verbs = pd.DataFrame(data_clean.transcript.apply(nouns_and_verbs))

# Create a new document-term matrix using only nouns and verbs
cvnv = CountVectorizer(stop_words=stop_words)
data_cvnv = cvnv.fit_transform(data_nouns_verbs.transcript)
data_dtmnv = pd.DataFrame(data_cvnv.toarray(), columns=cvnv.get_feature_names_out())
data_dtmnv.index = data_nouns_verbs.index

In [39]:
# Create the gensim corpus
corpusnv = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmnv.transpose()))

# Create the vocabulary dictionary
id2wordnv = dict((v, k) for k, v in cvnv.vocabulary_.items())

# Build a topic model using LDA with the new DTM that includes nouns and verbs
lda_nv = models.LdaModel(corpus=corpusnv, 
                         id2word=id2wordnv, 
                         num_topics=4, 
                         passes=120,
                         alpha='auto',  # Set alpha to 'auto' to improve model fitting
                        eta=0.01)

lda_nv.print_topics()

[(0,
  '0.028*"fucking" + 0.020*"went" + 0.013*"day" + 0.012*"fuck" + 0.011*"going" + 0.011*"thing" + 0.010*"guns" + 0.009*"goes" + 0.009*"ive" + 0.008*"theyre"'),
 (1,
  '0.007*"cause" + 0.006*"man" + 0.006*"gon" + 0.006*"mean" + 0.006*"way" + 0.006*"say" + 0.005*"want" + 0.005*"didnt" + 0.005*"good" + 0.005*"shit"'),
 (2,
  '0.007*"going" + 0.006*"say" + 0.006*"fucking" + 0.006*"little" + 0.005*"want" + 0.005*"went" + 0.005*"fcking" + 0.005*"thing" + 0.004*"didnt" + 0.004*"theyre"'),
 (3,
  '0.010*"gon" + 0.007*"shit" + 0.006*"hes" + 0.006*"theres" + 0.005*"guy" + 0.005*"theyre" + 0.005*"fucking" + 0.005*"cause" + 0.005*"guys" + 0.004*"thing"')]

Topic 0: Intense Experiences and Actions

Topic 1: Opinions and Explanations

Topic 2: Describing Events and Reactions

Topic 3: People and Conversations